### Model Candidate - Neural Network

Our objective is to beat the model that is being used by our client, Lisboa Bank

In [152]:
###Insert Baseline Model Performance Metrics Here

We will start by creating a simple neural network and seeing how it performs

In [153]:
%matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Using matplotlib backend: Qt5Agg


In [154]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn import preprocessing
#from keras import keras_metrics

In [155]:
#Loading df1 after it has been augmented in iteration 2:
df = pd.read_pickle('../data/pickle_files/df_pickle_w_all_stats')
#Excluding the duration variable as it cannot be used in our baseline
df = df.drop(columns = ['duration'])

In [156]:
df.drop(columns=['Date'], inplace=True)

In [157]:
def scale_noncat_only(df):
    X_temp_noncat = df.select_dtypes(exclude=['int8'])
    X_temp_cat = df.select_dtypes(include=['int8'])
    scaler = preprocessing.StandardScaler().fit(X_temp_noncat)
    X_transformed = scaler.transform(X_temp_noncat)
    X = pd.concat([pd.DataFrame(X_transformed, columns =X_temp_noncat.columns), X_temp_cat], axis = 1)
    X = X.drop(columns=['y'])
    y = df['y']
    #will work with numpy arrays
    y = np.array(y)
    X = np.array(X)
    
    return X, y

X, y = scale_noncat_only(df)

In [158]:
#Checking dtypes have loaded correctly (should return empty index)
df.select_dtypes(exclude = ['float64', 'int64', 'int8']).columns

Index([], dtype='object')

In [159]:
X.shape

(41188, 72)

In [160]:
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold

In [161]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 4)

In [162]:
model = Sequential()
#keras.layers.Dense(units, activation=None, use_bias=True, 
#kernel_initializer='glorot_uniform', bias_initializer='zeros', 
#kernel_regularizer=None, bias_regularizer=None, 
#activity_regularizer=None, kernel_constraint=None, bias_constraint=None)

In [163]:
model.add(Dense(72, input_dim=72, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(72, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [164]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [165]:
model.fit(X_train, y_train,
          epochs=50,
          batch_size=100, verbose=0)

In [166]:
score = model.evaluate(X_test, y_test, batch_size=100)

10297/10297 [==============================] - ETA:  - 5s 470us/step


In [167]:
print(model.metrics_names)

['loss', 'acc']


In [168]:
score

[0.28209459523477637, 0.8967660504069573]

Let's create more simple models and see how changing basic hyperparameters affects it

In [169]:
drop_out = 0.5

In [170]:
def create_simple_model():
    
    model = Sequential()
    
    model.add(Dense(64, input_dim=72, activation='relu'))
    model.add(Dropout(drop_out))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(drop_out))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])
    return model

In [171]:
#from keras.utils import plot_model
#plot_model(model, to_file='model.png')

In [172]:
def fit_model(epochs, batch_size, verbose, val_split):
    history = model.fit(X_train, y_train, epochs=epochs, validation_split = val_split,
                        batch_size=batch_size, verbose=verbose)
    return history

In [173]:
def plot_scores(history, val = False):
    
    # Plot training & validation accuracy values
    plt.plot(history.history['acc'])
    if val == True:
        plt.plot(history.history['val_acc'])
        
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()

In [174]:
def plot_loss(history, val = False):
    # Plot training & validation loss values
    plt.plot(history.history['loss'])
    
    if val == True:
        plt.plot(history.history['val_loss'])
    
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()

In [175]:
#Setting model parameters
drop_out = 0.5

In [176]:
model = create_simple_model()

In [177]:
history = fit_model(500, 1000, 0, None)

In [178]:
plot_scores(history)

In [179]:
plot_loss(history)

In [180]:
score = model.evaluate(X_test, y_test, batch_size=100)

10297/10297 [==============================] - 5s 491us/step


In [181]:
print(model.metrics_names)
score

['loss', 'acc']


[0.28294023889076875, 0.895309314804655]

After a few runs of this basic model with different values for units and optimizer, we are noticing that it tends to always perform at most with 90% accuracy ca.

Let's try to improve this first model with the following:
    - Validation
    - Higher Regularization
    - Random Weight Initialization
    - Hypterparameter Tuning

In [182]:
#Setting new drop out regularization term
drop_out = 0.7

In [183]:
model = create_simple_model()

In [184]:
history = fit_model(100, 100, 0, .2)

In [185]:
plot_scores(history, True)

In [186]:
plot_loss(history)

In [187]:
score = model.evaluate(X_test, y_test, batch_size=100)

10297/10297 [==============================] - 1s 117us/step


In [188]:
print(model.metrics_names)
score

['loss', 'acc']


[0.30747421872063585, 0.8988054762641886]

Evaluating Model with Stratified K Cross Validation

In [189]:
from keras.wrappers.scikit_learn import KerasClassifier

In [190]:
def create_base_model():
# create model
    model = Sequential()
    model.add(Dense(72, input_dim=72, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [191]:
estimator = KerasClassifier(build_fn=create_base_model, epochs=100, batch_size=100, verbose=0)

In [192]:
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=4)

In [ ]:
results = cross_val_score(estimator, X_train, y_train, cv=skf) #using parallel cores
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Now trying again with earlier simple model function, which actually produces a more complex neural network than the base model function

In [ ]:
estimator = KerasClassifier(build_fn=create_simple_model, epochs=100, batch_size=100, verbose=0)

In [ ]:
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=4)

In [ ]:
results = cross_val_score(estimator, X_train, y_train, cv=skf) #using parallel cores
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

New Round of CV using more complex network architecture:

In [ ]:
drop_out = 0.6

In [ ]:
def create_complex_model():
    model = Sequential()
    model.add(Dense(72, input_dim=72, activation='relu'))
    model.add(Dropout(drop_out))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(72, activation='relu'))
    model.add(Dropout(drop_out))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
estimator = KerasClassifier(build_fn=create_complex_model, epochs=100, batch_size=100, verbose=0)

In [ ]:
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=4)

In [ ]:
results = cross_val_score(estimator, X_train, y_train, cv=skf) #using parallel cores
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Additional Round of CV with different parameters

In [ ]:
drop_out = 0.3

In [ ]:
def create_complex_model():
    model = Sequential()
    model.add(Dense(72, input_dim=72, activation='relu'))
    model.add(Dropout(drop_out))
    model.add(Dense(12, activation='relu'))
    model.add(Dense(12, activation='relu'))
    model.add(Dropout(drop_out))
    model.add(Dense(72, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
estimator = KerasClassifier(build_fn=create_complex_model, epochs=100, batch_size=100, verbose=0)

In [ ]:
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=4)

In [ ]:
results = cross_val_score(estimator, X_train, y_train, cv=skf) #using parallel cores
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Testing Model with Hyperparameter Tuning

In [ ]:
#creating a new model function using the network architecture 
#that seemed to do best in the cross validation trials above
def create_new_model(optimizer='rmsprop', init='glorot_uniform'):
    #TBD
    model = Sequential()
    model.add(Dense(72, input_dim=72,kernel_initializer=init, activation='relu'))
    model.add(Dense(1, kernel_initializer=init,activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
estimator = KerasClassifier(build_fn=create_new_model, verbose=0)

In [ ]:
optimizer=['rmsprop', 'adam']
epochs=[300, 500, 700]
batch_size=[50,100,500]
inits = ['glorot_uniform', 'normal', 'uniform']

param_grid = dict(optimizer=optimizers, epochs=epochs, batch_size=batches, init=inits)
gs = GridSearchCV(estimator=estimator, param_grid=param_grid, n_jobs=-1)

In [ ]:
grid_result = gs.fit(X_train, y_train)

In [ ]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
print("%f (%f) with: %r" % (mean, stdev, param))